In [1]:
from typing import Generator
import utils.llm_utils as llm
import datasets # type: ignore

input_dataset = 'lmsys/chatbot_arena_conversations'
dataset = datasets.load_dataset(input_dataset)['train']
n_rows = len(dataset)
print(str(n_rows))

model_name = 'microsoft/Phi-3-mini-4k-instruct'
tokenizer, model = llm.load_tokenizer_and_model(model_name)

33000


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Test:
out_jsonl = "1_dataset_creator/phi3_arena_brief_dataset.jsonl"
with open(out_jsonl, 'w') as f: pass

dataset = dataset.select_columns(['question_id', 'conversation_a']) 
dataset = dataset.map(
        lambda example: {'question': example['conversation_a'][0]['content']})

for normal_chunk in llm.process_variable_chunks(dataset, tokenizer, model, 100, 1500, 50):
    normal_chunk = normal_chunk.select_columns(['question_id', 'question', 'answer'])
    with open(out_jsonl, 'ab') as f:
        normal_chunk.to_json(f, lines=True)
    break

In [2]:
import utils.utils as utils

# Word limits for normal and brief answers.
normal_word_limit = 120
brief_word_limit = 20
# A word is "between 5 and 6.5 characters per word including spaces and punctuation":
# https://charactercounter.com/characters-to-words
normal_max_ch_soft = normal_word_limit * 6
brief_max_ch_soft = brief_word_limit * 6
# Hard limit adds 20% buffer and divides by 4 to get LLM token limit.
# https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
normal_max_tokens_hard = int(normal_max_ch_soft * 1.2 / 4)
brief_max_tokens_hard = int(brief_max_ch_soft * 1.2 / 4)

normal_prompt = f'''Answer the user prompt below "---" line. Never exceed {normal_max_ch_soft} characters / {normal_word_limit} words.
---
'''

brief_prompt = f'''Given the user prompt and a normal answer, generate a brief answer. A brief answer should be as short as possible but still answer the question and give relevant information. Never exceed {brief_max_ch_soft} characters / {brief_word_limit} words.
Examples between --- lines:
--- Example 1 ---
Your input:
Question: How much is 2+3?
Normal answer: Expression 2+3 is equal to 5.
Your output:
5
--- Example 2 ---
Your input:
Question: What is the color of the sky?
Normal answer: The sky is blue.
Your output:
Blue
--- End of examples

Considering all the above, give a brief answer to the prompt and normal answer below:
'''

def brief_prompt(example):
    q = example['question']
    na = example['Answer-normal']
    return f"{brief_prompt}Question: {q}\nNormal answer: {na}"

out_jsonl = "1_dataset_creator/phi3_arena_brief_dataset.jsonl"
with open(out_jsonl, 'w') as f: pass

normal_dataset = dataset.select_columns(['question_id', 'conversation_a']) 
normal_dataset = normal_dataset.map(
        lambda example: {'question': example['conversation_a'][0]['content']}) 
normal_dataset = normal_dataset.map(
        lambda example: {'prompt': normal_prompt + example['question']})

timer = utils.Timer(f"Processing first 100...")
with timer:
    for normal_chunk in llm.process_variable_chunks(normal_dataset, tokenizer, model, 100, 5000, normal_max_tokens_hard):
        normal_chunk = normal_chunk.select_columns(['question_id', 'question', 'answer'])
        normal_chunk = normal_chunk.rename_column('answer', 'Answer-normal')
        normal_chunk = normal_chunk.map(
            lambda example: {'prompt':  brief_prompt(example)})
        
        for brief_chunk in llm.process_variable_chunks(normal_chunk, tokenizer, model, 100, 5000, brief_max_tokens_hard):
            brief_chunk = brief_chunk.select_columns(['question_id', 'question', 'answer', 'Answer-normal'])
            brief_chunk = brief_chunk.rename_column('answer', 'Answer-short')
            with open(out_jsonl, 'ab') as f:
                brief_chunk.to_json(f, lines=True)
        
        break

print(f"Time/sample: {timer.last_elapsed_time/100:.2f} sec") 

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Processing first 100...


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...total time: 53.63 sec
Time/sample: 0.54 sec
